In [1]:
import pandas as pd

In [2]:
df_thread = pd.read_csv('thread_18cff0b946f3a4d4_cleaned.csv', index_col='Unnamed: 0')
df_thread.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11 entries, 76 to 100
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   threadId          11 non-null     object
 1   from              11 non-null     object
 2   date              11 non-null     object
 3   body_cleaned      11 non-null     object
 4   body_cleaned_len  11 non-null     int64 
dtypes: int64(1), object(4)
memory usage: 528.0+ bytes


In [3]:
df_thread = df_thread.reset_index().drop('index', axis=1)
df_thread

,threadId,from,date,body_cleaned,body_cleaned_len
0,18cff0b946f3a4d4,Felipe dos Santos Giacomel <felipe.giacomel@tr...,2024-01-12 15:59:44-03:00,"Prezada DGBD, boa tarde!\r\n\r\nEstamos com er...",105
1,18cff0b946f3a4d4,Edson Elnei Lehr <elehr@trt4.jus.br>,2024-01-12 16:29:16-03:00,Copiando para o e-mail correto da DIBD.\r\n\r\...,15
2,18cff0b946f3a4d4,Edson Elnei Lehr <elehr@trt4.jus.br>,2024-01-12 17:09:03-03:00,Parece ser de tnsnames.ora desatualizado na es...,36
3,18cff0b946f3a4d4,Felipe dos Santos Giacomel <felipe.giacomel@tr...,2024-01-12 17:29:30-03:00,"Obrigado, Edson.",2
4,18cff0b946f3a4d4,Evandro Bassanesi <evandro.bassanesi@trt4.jus.br>,2024-01-12 17:34:50-03:00,"Felipe,\r\n\r\nEste está desatualizado. O banc...",24
5,18cff0b946f3a4d4,Felipe dos Santos Giacomel <felipe.giacomel@tr...,2024-01-12 17:46:45-03:00,Vamos atualizar. Obrigado!\r\n\r\n\r\nAtencios...,18
6,18cff0b946f3a4d4,Paulo Ricardo Ritter <pritter@trt4.jus.br>,2024-01-15 08:58:54-03:00,"Felipe, isso não ocorre no servidor do Markvis...",104
7,18cff0b946f3a4d4,Paulo Ricardo Ritter <pritter@trt4.jus.br>,2024-01-15 09:19:39-03:00,"Agora que entendi, o problema que refere, não...",46
8,18cff0b946f3a4d4,Paulo Ricardo Ritter <pritter@trt4.jus.br>,2024-01-15 10:18:12-03:00,A GPO de atualização do TNSnames (desktop-all-...,28
9,18cff0b946f3a4d4,Francisco Flesch <francisco.flesch@trt4.jus.br>,2024-01-15 11:05:28-03:00,Eu troquei o TNSnames na última vez que deu pr...,41


In [23]:
from langchain_openai import ChatOpenAI, OpenAI

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

llm_json = OpenAI(
  temperature= 0,
  modelName= "gpt-4o-mini",
  modelKwargs = {
    'response_format': { 'type': "json_object" },
  },
)

llm_json2 = ChatOpenAI(model="gpt-4o-mini").bind(
    response_format={"type": "json_object"}
)

/workspaces/agent_email_kb/.venv/lib/python3.10/site-packages/langchain_core/utils/utils.py:225: UserWarning: WARNING! modelName is not default parameter.
                modelName was transferred to model_kwargs.
                Please confirm that modelName is what you intended.
  warnings.warn(
/workspaces/agent_email_kb/.venv/lib/python3.10/site-packages/langchain_core/utils/utils.py:225: UserWarning: WARNING! modelKwargs is not default parameter.
                modelKwargs was transferred to model_kwargs.
                Please confirm that modelKwargs is what you intended.
  warnings.warn(


In [2]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(
    separator = "\n\n",  # Split character (default \n\n)
    chunk_size=100,
    chunk_overlap=1,
    length_function=len,
)

In [8]:
text = """
Prezada DGBD, boa tarde!

Estamos com erro ao fazer operações na base do Infor de São Leopoldo. Podem
dar uma olhada, por favor?


[2024-01-12 11:41:18] Falha ao conectar … base do inFOR de SÆo Leopoldo 1¦
> VT (SAOLEO.WORLD) com o usu rio inforadm: Exception calling ""Open"" with ""0""
> argument(s): ""ORA-12541: TNS: nÆo h  listener""




Atenciosamente,

Felipe Giacomel
Divisão de Microinformática
TRT4 - Secretaria de Tecnologia da Informação e Comunicações

-- 
You received this message because you are subscribed to the Google Groups ""SETIC-DIM- Divisao de Microinformatica"" group.
To unsubscribe from this group and stop receiving emails from it, send an email to setic.dim-lista+unsubscribe@trt4.jus.br.
"""

In [9]:
splits = splitter.split_text(text)

Created a chunk of size 102, which is longer than the specified 100
Created a chunk of size 208, which is longer than the specified 100
Created a chunk of size 104, which is longer than the specified 100


In [10]:
strings = { k:v for (k,v) in zip(range(len(splits)), splits) }
strings

{0: 'Prezada DGBD, boa tarde!',
 1: 'Estamos com erro ao fazer operações na base do Infor de São Leopoldo. Podem\ndar uma olhada, por favor?',
 2: '[2024-01-12 11:41:18] Falha ao conectar … base do inFOR de SÆo Leopoldo 1¦\n> VT (SAOLEO.WORLD) com o usu rio inforadm: Exception calling ""Open"" with ""0""\n> argument(s): ""ORA-12541: TNS: nÆo h  listener""',
 3: 'Atenciosamente,',
 4: 'Felipe Giacomel\nDivisão de Microinformática\nTRT4 - Secretaria de Tecnologia da Informação e Comunicações',
 5: '-- \nYou received this message because you are subscribed to the Google Groups ""SETIC-DIM- Divisao de Microinformatica"" group.\nTo unsubscribe from this group and stop receiving emails from it, send an email to setic.dim-lista+unsubscribe@trt4.jus.br.'}

In [47]:
prompt_grader_importancy_system = """
    You are a helpful assistant who is in charge of classifying strings as either interesting to a use case or not.
    This strings will be used by another llm to generate a report and a knowledge base fo a IT company.
    So the use case is to determine if the string contain helpful information to create a knowledge base or not.

    You should output a json. Key shold be the key of the string, the value should be the boolean value
"""


prompt_grader_importancy_user = """
    {STRINGS}
"""

from langchain_core.prompts import ChatPromptTemplate

prompt_grader = ChatPromptTemplate.from_messages(
    [("system", prompt_grader_importancy_system), ("user", prompt_grader_importancy_user)]
)

In [48]:
# chain_grader = prompt_grader | llm
chain_grader = prompt_grader | llm_json2

In [35]:
response = chain_grader.invoke({"strings": strings})
strings_graded = response.content

In [36]:
strings_graded

'{\n    "0": false,\n    "1": true,\n    "2": true,\n    "3": false,\n    "4": false,\n    "5": false\n}'

In [37]:
import json

strings_graded = json.loads(strings_graded)
strings_graded

{'0': False, '1': True, '2': True, '3': False, '4': False, '5': False}

In [38]:
strings_graded_filtered = [ strings[int(k)] for (k,v) in strings_graded.items() if v == True ]
strings_graded_filtered

['Estamos com erro ao fazer operações na base do Infor de São Leopoldo. Podem\ndar uma olhada, por favor?',
 '[2024-01-12 11:41:18] Falha ao conectar … base do inFOR de SÆo Leopoldo 1¦\n> VT (SAOLEO.WORLD) com o usu rio inforadm: Exception calling ""Open"" with ""0""\n> argument(s): ""ORA-12541: TNS: nÆo h  listener""']

In [44]:
prompt_report_creator_system = """
    You are an expert newswriter that write email conversation reports.
    You dont need to give any IT support, You should just write the report like a newswriter would write a news.
    You will receive two inputs: a knowledge base document of a IT company and a list of strings extracted from a conversation between IT Technicians.
    You should consolidate the strings into the knowledge base document.
    Some strings should by copied char by char, like log strings, error messages.
    The language of the conversation is generaly in Brazilian Portuguese. So your report should be in Brazilian Portuguese.
    Dont write uninformative phrases like "relatorio de conversa".
    If the previous report is empty, just create a new one from scratch.
    You should output a technical report structured like a book.
    
    Do not discuss the email itself, focus rewriting the content
    Do not sugest any solution or output any information that are not in the strings or in the previous report.

    Dont write like: "O email fala sobre um problema tal e bla bla.". 
    You should be more directly to point.

    Sections of the report:
            - subject or description of the problem
            - technologys: the name of the technologys being addressed
            - Description: rewrite all the strings in order in a logical, elegant and techical way. 
                    In the case of instructions with steps on how to do something you should rewrite it. Dont loose anyinformation. 
                    If it exists, a exact description char by char of the error message that happened. you should just copy system messages necessary to address the problem.
            - Solution: if mentioned in the strings, create a summary of the solution presented. You could update the solution with new information if possible. 
                    Dont suggest any if there isnt in the strings or in te privous report. Your task is not to suggest any solution, your task is to report what is in the report and the strings.
    """

prompt_report_creator_user = """
    PREVIOUS REPORT BELOW
    {REPORT}
    
    STRINGS TO BE CONSOLIDATED BELOW
    {STRINGS}
"""

from langchain_core.prompts import ChatPromptTemplate

prompt_report_creator = ChatPromptTemplate.from_messages(
    [("system", prompt_report_creator_system), ("user", prompt_report_creator_user)]
)

In [45]:
chain_report_creator = prompt_report_creator | llm

In [46]:
response = chain_report_creator.invoke( {'REPORT': '', 'STRINGS': "\n\n".join(strings_graded_filtered)} )

print( response.content )

# Relatório Técnico

## Descrição do Problema
Erro ao realizar operações na base do Infor de São Leopoldo.

## Tecnologias
Infor, Oracle Database

## Descrição
Foi reportado um erro ao tentar realizar operações na base do Infor de São Leopoldo. O log do sistema apresenta a seguinte mensagem de erro:

```
[2024-01-12 11:41:18] Falha ao conectar … base do inFOR de SÆo Leopoldo 1¦
> VT (SAOLEO.WORLD) com o usu rio inforadm: Exception calling ""Open"" with ""0""
> argument(s): ""ORA-12541: TNS: nÆo h  listener""
```

## Solução
Nenhuma solução foi mencionada nas strings apresentadas.


In [49]:
def test_prompt(old_report, df_row):
    index, row = df_row
    email = row['body_cleaned']
    
    print( "--------| OLD_REPORT" )
    print( old_report )
    print( "------------------------------------------------------------------------------------------" )

    print( "--------| EMAIL" )
    print( email )
    print( "------------------------------------------------------------------------------------------" )

    # split email
    splits = splitter.split_text(email)
    strings = { k:v for (k,v) in zip(range(len(splits)), splits) }

    # remove all uninformative strings
    response = chain_grader.invoke( { 'STRINGS': strings } )
    strings_graded = response.content

    # parse json
    strings_graded = json.loads(strings_graded)

    # get useful strings
    strings_graded_filtered = [ strings[int(k)] for (k,v) in strings_graded.items() if v == True ]

    response = chain_report_creator.invoke( {'REPORT': old_report, 'STRINGS': "\n\n".join(strings_graded_filtered)} )
    new_report = response.content
    
    print( "--------| NEW_REPORT" )
    print( new_report )
    print( "------------------------------------------------------------------------------------------" )

    # response = chain_report_revisor.invoke({'REPORT': new_report, 'EMAIL_CONTENT': email})
    # list_of_informations = response.content

    # print( "--------| LIST_OF_INFORMATIONS" )
    # print( list_of_informations )
    # print( "------------------------------------------------------------------------------------------" )

    # response = chain_combine.invoke({'REPORT': new_report, 'LIST_OF_INFORMATIONS': list_of_informations})
    # report_combined = response.content

    # print( "--------| REPORT_COMBINED" )
    # print( report_combined )
    # print( "------------------------------------------------------------------------------------------" )
        
    # final_report = report_combined
    final_report = new_report
    # return old_report, email, new_report, list_of_informations, final_report
    return old_report, email, new_report, final_report


In [54]:
df_iter = df_thread.iterrows()
final_report = ""
reports = []

In [56]:
old_report, email, new_report, final_report = test_prompt(final_report, next(df_iter))
reports.append( ( old_report, email, new_report, final_report ) )

--------| OLD_REPORT
# Relatório Técnico

## Descrição do Problema
Erro ao realizar operações na base do Infor de São Leopoldo.

## Tecnologias
- Infor
- Banco de Dados
- TNS (Transparent Network Substrate)

## Descrição
Foi reportado um erro ao tentar conectar à base do Infor de São Leopoldo. A mensagem de erro gerada foi:

```
[2024-01-12 11:41:18] Falha ao conectar … base do inFOR de SÆo Leopoldo 1¦
> VT (SAOLEO.WORLD) com o usu rio inforadm: Exception calling "Open" with "0"
> argument(s): "ORA-12541: TNS: nÆo h  listener"
```

O erro indica que não há um listener disponível para a conexão com a base de dados.

## Solução
Não foram apresentadas soluções específicas nas strings fornecidas.
------------------------------------------------------------------------------------------
--------| EMAIL
Copiando para o e-mail correto da DIBD.

At.te,
*-----------*
*Edson Lehr*
((51)3255-2491
* elehr@trt4.jus.br
DIBD/SETIC/TRT4
-----------------------------------------------------------------

In [ ]:
fazer o reort creator fazr output de json e fazer uma chave pro texto e outra pra solução.
tem q ser assim se nao o criador pega a solucao do old relatorio e acha q foi alguem q sugeriu e começa a entrar um loop nisso.
